## Logistic Regression Problem Statement

- Iris dataset

In [5]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [6]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [7]:
X, y = datasets.load_iris(return_X_y=True)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [9]:
params = {
    "solver": "lbfgs",
    "penalty": "l2",
    "random_state": 42,
    "max_iter": 100
}

In [10]:
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [11]:
y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.900


In [12]:
mlflow.set_experiment("MLflow Quickstart")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.set_tag("training_info", "Basic logistic regression model for Iris dataset")

2025/08/27 20:02:24 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Quickstart' does not exist. Creating a new experiment.


🏃 View run crawling-grub-899 at: http://127.0.0.1:5000/#/experiments/1/runs/3d167543015946f79b6f42e48531bc1b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [13]:
signature = infer_signature(X_train, lr.predict(X_train))

In [14]:
mlflow.sklearn.log_model(
    sk_model=lr,
    name="iris_model",
    signature=signature,
    input_example=X_train[:5],
    registered_model_name="tracking_quickstart"
)

Successfully registered model 'tracking_quickstart'.
2025/08/27 20:02:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking_quickstart, version 1
Created version '1' of model 'tracking_quickstart'.


In [15]:
params_new = {
    "solver": "newton-cg",
    "random_state": 1000,
    "max_iter": 100
}

lr_new = LogisticRegression(**params_new)
lr_new.fit(X_train, y_train)
y_pred_new = lr_new.predict(X_test)
accuracy_new = accuracy_score(y_test, y_pred_new)

with mlflow.start_run():
    mlflow.log_params(params_new)
    mlflow.log_metric("accuracy", accuracy_new)
    mlflow.set_tag("training_info", "Logistic regression with newton-cg solver")
    signature_new = infer_signature(X_train, lr_new.predict(X_train))
    mlflow.sklearn.log_model(
        sk_model=lr_new,
        name="iris_model_new",
        signature=signature_new,
        input_example=X_train[:5],
        registered_model_name="tracking_quickstart"
    )

Registered model 'tracking_quickstart' already exists. Creating a new version of this model...
2025/08/27 20:03:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking_quickstart, version 2
Created version '2' of model 'tracking_quickstart'.


🏃 View run invincible-boar-203 at: http://127.0.0.1:5000/#/experiments/1/runs/44c2b159ff694cdabefaaa8f0963870f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


## Understanding the Model Artifacts generated by MLFlow

### Inferencing and Validating Model

In [4]:
from mlflow.models import validate_serving_input

model_uri = "runs:/5e3b1782a6b0403997a56b4e3e337a0e/iris_model"
serving_payload = {"inputs": [[5.1, 3.5, 1.4, 0.2], [4.9, 3.0, 1.4, 0.2], [6.2, 3.4, 5.4, 2.3], [5.9, 3.0, 5.1, 1.8]]}

validate_serving_input(model_uri, serving_payload)

MlflowException: Run '5e3b1782a6b0403997a56b4e3e337a0e' not found

In [ ]:
import mlflow.pyfunc

loaded_model = mlflow.pyfunc.load_model(model_uri)
prediction = loaded_model.predict(X_test)

In [ ]:
import pandas as pd
from sklearn import datasets

iris_feature_names = datasets.load_iris().feature_names
result = pd.DataFrame(X_test, columns=iris_feature_names)
result['actual_class'] = y_test
result['predicted_class'] = prediction

In [ ]:
result[:5]